In [ ]:
import urllib 
import concurrent.futures
import threading
import psycopg2
import time
import hashlib
import socket
from urllib.parse import urlparse
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from bs4 import BeautifulSoup

WEB_DRIVER_LOCATION = "./geckodriver"
TIMEOUT = 5

def calculate_hash(data):
    hash_object = hashlib.sha256()
    hash_object.update(data.encode('utf-8'))
    hash_hex = hash_object.hexdigest()
    return hash_hex

frontier = ["https://vreme.arso.gov.si", "https://spot.gov.si"]
link_history = []
current_hosts = set()
firefox_options = FirefoxOptions()
firefox_options.add_argument("user-agent=fri-ieps-TEST")
#, "https://spot.gov.si", "https://e-uprava.gov.si/"]

def shave_url(url):
    spl = url.split('/') #"https://vreme.arso.gov.si/napoved -> vreme.arso.gov.si
    if len(spl) > 2:
        return spl[2]
    return url

def crawl_page(n, thread):
    for _ in range(n):
        counter = 0
        frontier_len = 0

        driver = webdriver.Firefox(executable_path=WEB_DRIVER_LOCATION, options=firefox_options)
        host = ""
        hostname = ""
        with lock:
            counter = 0
            if len(frontier) > 0:
                url = ""
                for i in range(len(frontier)): # iščemo stran na IP-ju, ki še ni zaseden
                    url = frontier[i]
                    if url in link_history:
                        continue
                    try:
                        hostname = urlparse(url).hostname
                        host = socket.gethostbyname(hostname)
                        if host not in current_hosts:
                            current_hosts.add(host)
                            frontier.pop(i)
                            break
                        print("Server on " + url + " busy (" + host + ")")
                    except:
                        print("Wrong URL format")
                        continue
                link_history.append(url)
                
                print(f"Retrieving web page URL '{url}' ({host}) - thread {thread}")
                driver.get(url)
            else:
                print(f'----------------empty frontier---------------- thread {thread}')
                continue
     
        time.sleep(TIMEOUT)
        current_hosts.remove(host)
        
        html = driver.page_source
        #page_msg = driver.find_element(By.CSS_SELECTOR, ".panel-header")
        #print(f"Web page message: '{page_msg.text}'")
        driver.close()

        soup = BeautifulSoup(html)
        global_links = []
        domain_links = []
        with lock:
            for link in soup.find_all('a'):
                link = link.get('href')
                if link is not None:
                    if link.startswith('/'):
                        link = "http://" + hostname + link
                        if link not in frontier:
                            frontier.append(link)
                    elif link.startswith('http') and 'gov.si' in link:
                        global_links.append(link)
                        if link not in frontier:
                            frontier.append(link)
    if thread == 0:
        print("Link history: ")
        for l in link_history:
            print(l)
        print("Frontier: ")
        for l in frontier:
            print(l)
        
lock = threading.Lock()
database_port = 5431
def get_db():
    conn = psycopg2.connect(host="localhost", port = database_port, user="user", password="SecretPassword")
    conn.autocommit = True

    cur = conn.cursor()
    cur.execute("SELECT * FROM crawldb.site")
    cur.close()
    conn.close()
    

def para(num_workers, pages_per_thread):
    with concurrent.futures.ThreadPoolExecutor(max_workers=num_workers) as executor:
        print(f"\n ... executing workers ...\n")
        for i in range(num_workers):
            executor.submit(crawl_page, pages_per_thread, i)
para(3, 5)

    